## IDs:
ID 1: 204502926 <br>
ID 2: 039065313

## Imports:

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
folder='./ml-100k/'

In [18]:
item_col = ['item_id','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation',
              'Children','Comedy','Crime','Documentary','Drama','Fantasy',
              'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movie_df = pd.read_csv(folder+"u.item",sep='|',encoding='ISO-8859-1',names=item_col)

# column_names = ['user_id', 'item_id', 'rating', 'timestamp']
# ratings = pd.read_csv(folder+'ua.base',sep='\t',names=column_names) 
# sep cannot infer '\t' from files so explicitly supply arg
# ratings.info()

names = ['user_id', 'item_id', 'rating', 'timestamp']
folder = "./ml-100k/"
data_df = pd.read_csv(folder+'u.data', sep='\t', names=names)
data_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
# pd.pivot_table(data_df, values='rating', index=['item_id'], columns = ['user_id']).fillna(0)

In [24]:
n_users = data_df.user_id.unique().shape[0]
n_items = data_df.item_id.unique().shape[0]
ratings = np.zeros((n_users, n_items))
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [25]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    train = ratings.copy()

    for user in range(ratings.shape[0]):
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=10, 
                                        replace=False)
        train[user, test_ratings] = 0.
        test[user, test_ratings] = ratings[user, test_ratings]
        
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test

train_ratings, test_ratings = train_test_split(ratings)

### Question 1: Item Similarity (40 points)

#### a. Build a movie profile vector based on the item features of your choice. Suggested options: item genres

In [8]:
movie_profile_df = movie_df[['item_id','Action','Adventure','Animation',
              'Children','Comedy','Crime','Documentary','Drama','Fantasy',
              'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']].set_index('item_id')
movie_profile_df.sort_index(axis=0, inplace=True)
movie_profile_df.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item_id,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


#### b. Build a function which provides the 5 most similar items to a specific item. Item to item similarity is calculated using the cosine similarity metric.

In [9]:
cosine_similarity_item_item =cosine_similarity(movie_profile_df, movie_profile_df)

In [13]:
def check_for_similarity(item_id, item_ids_to_compare):
    flag = True
    
    for item_id_to_compare in item_ids_to_compare:
        for ((columnNameA, columnDataA), (columnNameB, columnDataB)) in zip(movie_profile_df.loc[[item_id]].iteritems(), movie_profile_df.loc[[item_id_to_compare]].iteritems()):
        #     print(f'Colunm Name: {columnNameA} genere: {columnDataA.values}, {columnDataB.values}')
            if columnDataA.values != columnDataB.values:
                print(f'Colunm Name: {columnNameA}, {item_id}: {columnDataA.values}, {item_id_to_compare}: {columnDataB.values}')
                flag = False
                break
    #             
    return flag

In [78]:
def predict_most_similar_items_per_movie(cosine_similarity_matrix, profile_A_df, profile_B_df, item_id, num_items=5, exclude_if_item_match=True):
    result = np.argsort(cosine_similarity_matrix[profile_A_df.index.get_loc(item_id),:])[::-1][:num_items+1]
    if exclude_if_item_match:
        ret_result = [profile_B_df.index[i] for i in result if profile_B_df.index[i] != item_id] 
    else:
        ret_result = [profile_B_df.index[i] for i in result] 
    
    if len(ret_result) > num_items:
        ret_result = ret_result[:num_items]
    return ret_result

predict_most_similar_items_per_movie(cosine_similarity_item_item, movie_profile_df, movie_profile_df, 5)

[348, 1213, 329, 332, 1208]

#### c. Use the above function to find most similar items for 2 items from the dataset. Explain your results.

In [70]:
movie_id_1 = 5
movie_id_2 = 2

result_for_movie_id_1 = predict_most_similar_items_per_movie(cosine_similarity_item_item, movie_profile_df, movie_profile_df, movie_id_1)

result_for_movie_id_2 = predict_most_similar_items_per_movie(cosine_similarity_item_item, movie_profile_df, movie_profile_df, movie_id_2)

print(movie_id_1, result_for_movie_id_1, check_for_similarity(movie_id_1, result_for_movie_id_1))
print(movie_id_2, result_for_movie_id_2, check_for_similarity(movie_id_2, result_for_movie_id_2))

# lets see a case of mismatch:
print(movie_id_2, 1000, check_for_similarity(movie_id_2, [1000]))

# explain: cosin similarity calcs the cos value between the two vectors. because the vectors are binary.
# the result that would match the most are the one that have the exact same genres

5 [348, 1213, 329, 332, 1208] True
2 [118, 827, 1016, 1013, 566] True
Colunm Name: Action, 2: [1], 1000: [0]
2 1000 False


#### d. Use the matrix factorization item representation you built in exercise 1 to find the most similar items for the same 2 items you used above. Compare between the results.

***
### Question 2: Content based recommendation (60 points)

#### a. Build a function which recommends 5 most relevant items to a user. 

In [19]:
#Binary option should be set to True if the rating should be binary. 
#It should be set to True for the first approach and False for the second approach
BINARY_OPTION = False
#NEGATIVE WEIGHT is relevant only for the first approach
NEGATIVE_WEIGHT = 0.25

def brating(row):
    if row['rating'] >= 3:
        val = 1
    elif row['rating'] >=0:
        val = -NEGATIVE_WEIGHT
    else:
        val = row['rating']
    return val


ratings['binary_rating'] = ratings.apply(brating, axis=1)

if BINARY_OPTION:
    rating_column = 'binary_rating'
else:
    rating_column = 'rating'

In [27]:
movie_profile_df

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item_id,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [79]:
# user profile
user_id_1 = 900

# train
train_user_x_movie_df = pd.pivot_table(train_ratings, values=rating_column, index=['item_id'], columns = ['user_id'])
train_user_x_movie_df.sort_index(axis=0, inplace=True)
train_userIDs = train_user_x_movie_df.columns
train_user_profile_df = pd.DataFrame(columns = movie_profile_df.columns)

for i in range(len(train_user_x_movie_df.columns)):
    working_df = movie_profile_df.mul(train_user_x_movie_df.iloc[:,i], axis=0)
    train_user_profile_df.loc[train_userIDs[i]] = working_df.mean(axis=0)
    
train/test    

# # tests    
# test_user_x_movie_df = pd.pivot_table(test_ratings, values=rating_column, index=['item_id'], columns = ['user_id'])
# test_user_x_movie_df.sort_index(axis=0, inplace=True)
# test_userIDs = test_user_x_movie_df.columns
# test_user_profile_df = pd.DataFrame(columns = movie_profile_df.columns)

# for i in range(len(user_x_movie_df.columns)):
#     working_df = movie_profile_df.mul(user_x_movie_df.iloc[:,i], axis=0)
#     test_user_profile_df.loc[test_userIDs[i]] = working_df.mean(axis=0)
    
    

cosine_similarity_user_item = cosine_similarity(train_user_profile_df, movie_profile_df)
predict_most_similar_items_per_movie(cosine_similarity_user_item, train_user_profile_df, test_movie_profile_df, user_id_1, exclude_if_item_match=False)




[337, 1526, 311, 1424, 517]

In [ ]:
test_movie_profile_df = the ten movies that we deleted the rating for them from the rating matrix


In [ ]:
user_profile_df

In [73]:
movie_id_1

5

In [ ]:
[337, 1526, 311, 1424, 517]

In [84]:
for (c, v) in user_profile_df.loc[[900]].iteritems():
    print(c, v.values)

Action [0.2]
Adventure [0.14285714]
Animation [0.]
Children [0.]
Comedy [0.65714286]
Crime [0.22857143]
Documentary [0.]
Drama [1.4]
Fantasy [0.]
Film-Noir [0.05714286]
Horror [0.14285714]
Musical [0.14285714]
Mystery [0.25714286]
Romance [0.57142857]
Sci-Fi [0.]
Thriller [0.68571429]
War [0.51428571]
Western [0.11428571]


In [81]:
movie_profile_df.loc[[337]]

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
item_id,,,,,,,,,,,,,,,,,,
337,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0


In [98]:
user_values = []
for (movie_g, movie_v), (user_g, user_v) in zip(movie_profile_df.loc[[337]].iteritems(), user_profile_df.loc[[900]].iteritems()):
    user_values.append((user_g, float(user_v.values[0])))

user_values_sorted = sorted(user_values, key=lambda x: x[1], reverse=True)
user_values_sorted

[('Drama', 1.4), ('Thriller', 0.6857142857142857), ('Comedy', 0.6571428571428571), ('Romance', 0.5714285714285714), ('War', 0.5142857142857142), ('Mystery', 0.2571428571428571), ('Crime', 0.22857142857142856), ('Action', 0.2), ('Adventure', 0.14285714285714285), ('Horror', 0.14285714285714285), ('Musical', 0.14285714285714285), ('Western', 0.11428571428571428), ('Film-Noir', 0.05714285714285714), ('Animation', 0.0), ('Children', 0.0), ('Documentary', 0.0), ('Fantasy', 0.0), ('Sci-Fi', 0.0)]

In [102]:
for (movie_g, movie_v), (user_g, user_v) in zip(movie_profile_df.loc[[517]].iteritems(), user_profile_df.loc[[900]].iteritems()):
    print(f'Colunm Name: {movie_g} {movie_v.values[0]}, {user_v.values[0]}')

Colunm Name: Action 0, 0.2
Colunm Name: Adventure 0, 0.14285714285714285
Colunm Name: Animation 0, 0.0
Colunm Name: Children 0, 0.0
Colunm Name: Comedy 1, 0.6571428571428571
Colunm Name: Crime 0, 0.22857142857142856
Colunm Name: Documentary 0, 0.0
Colunm Name: Drama 1, 1.4
Colunm Name: Fantasy 0, 0.0
Colunm Name: Film-Noir 0, 0.05714285714285714
Colunm Name: Horror 0, 0.14285714285714285
Colunm Name: Musical 0, 0.14285714285714285
Colunm Name: Mystery 0, 0.2571428571428571
Colunm Name: Romance 1, 0.5714285714285714
Colunm Name: Sci-Fi 0, 0.0
Colunm Name: Thriller 0, 0.6857142857142857
Colunm Name: War 0, 0.5142857142857142
Colunm Name: Western 0, 0.11428571428571428


In [ ]:
# go over the movies and see that the genere 

### b. Test your recommender system on 2 users. Explain your results.

### c. Use the MRR metric to evaluate your recommender system on the test set. Use a cutoff value of 5.

In [103]:
MAX_COLUMNS_CUTOFF = 5
RATING_THRESHOLD_VALUE = 3


def get_mrr(pred, actual):    
    """
    Mean Reciprocal Rank
    
    Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105
    >>> rs = np.array([[0, 0, 0], [0, 1, 0], [1, 0, 0]])
    >>> mean_reciprocal_rank(rs)
    0.5
    >>> rs = [[0, 0, 0, 1], [1, 0, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.75
    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    matrix = get_relevant_item_matches_matrix_per_user(pred, actual)
    matrix = (np.asarray(r).nonzero()[0] for r in matrix)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in matrix])


def sort_user_ratings(row):
    '''Find the indices of the maximal values in given array'''
    return row.argsort()[-5:][::-1]


def get_relevant_item_matches_matrix_per_user(pred_matrix, actual_matrix):
    top_values_indices = np.apply_along_axis(sort_user_ratings, 1, actual_matrix)
    result = np.zeros_like(top_values_indices)
    rows = len(top_values_indices)
    columns = np.minimum(len(top_values_indices[0]), ≈)
    
    for i in range(rows):
        for j in range(columns):
            match = actual_matrix[i][top_values_indices[i][j]] >= RATING_THRESHOLD_VALUE and pred_matrix[i][top_values_indices[i][j]] >= RATING_THRESHOLD_VALUE

            if match:
                result[i][j] = 1
                break

    return result

get_mrr()

#### d. Compare the results of the content based recommender system to the matrix factorization recommender system using the same train and test set. Use the MRR metric for the comparison. 

#### e. Discuss the advantages and disadvantages of matrix factorization vs. content based recommender systems (refer to accuracy, complexity, explainaiblity and new user\new items) and when will you recommend using each approach. 